# BreastMammaryTissueJunctionAnalysis as a Notebook 

rMATS 3.2.5 was run on controlled access RNASeq files retrieved experiments stored in the Sequence Read Archive with controlled access managed by dbGaP.   The data were generated under the Gene Tissue Expression.

## rMATS RNASeq-MATS.py produces 10 different output types which get assembled into as type junction ID by sample ID matrices

### Alternative Splice Site Types are: (se, a3ss, a5ss, mxe, ri)

 This is input as ARGV1 into variable 'astype'

  * Skipped Exon events (se),
  * Alternative 3' splice site (a3ss),
  * Alternative 5' splice site (a5ss),
  * Mutually exclusive exon (mxe),
  * and retention intron (ri)

### There are two different kinds of junction counts

  * jc = junction counts - reads that cross the junction
  * jcec = junction counts plus reads on the target (such as included exon

### And the count type -- there are 5 types

  * inclusion levels (percent spliced in)
  * included junction counts (ijc)
  * skipped junction counts (sjc)
  * inclusion length (inclen)
  * skipped length (skiplen)

### function: fit_iso_tissue 

fit_iso_tissue expects the following input:

  * the tissue of interest (SMSTD) 
  * an ordered_merged_rmats -- which will be ordered to fit the count matrix
  * count matrix (inc or ijc & sjc merged)
  * splice type (a3ss, a5ss, mxe, ri or se)
  * junction_count type (jc or jcec)
  * count type (inc or the merged ijc,sjc)
  
### reordering to match annotations between count matrix and annotation matrix

Common problem is to match specifically the rows of an annotation matrix with the columns of a count matrix
`match` is the function that gives the re-ordering index required to accomplish this


## **NOTE**:

We assume that you have cloned the analysis repository and have `cd` into the parent directory. Before starting with the analysis make sure you have first completed the dependencies set up by following the instructions described in the **`dependencies/README.md`** document. All paths defined in this Notebook are relative to the parent directory (repository). Please close this Notebook and start again by following the above guidelines if you have not completed the aforementioned steps.

## Loading dependencies

In [2]:
library(limma)
library(multtest)
library(Biobase)
library(edgeR)
library(tibble)
library(R.utils)
# Install this version: > devtools::install_github("ropensci/piggyback@87f71e8", upgrade="never")
library(piggyback)

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following object is masked from ‘package:limma’:

    plotMA


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min


Loading required package: Biobase

Welcom

# Retrieving the results from the rMATS analysis using [`ropensci/piggyback`](https://github.com/ropensci/piggyback)

This notebook requires as input data the aggregated matrices from rMATS. We have archived the results from the Nextflow pipeline that generates the results using the method described by the author of the R package [`ropensci/piggyback`](https://github.com/ropensci/piggyback). We use the release named `rMATS.3.2.5.GTEx.V8.final_matrices` which can be accessed at [adeslatt/sbas_test/releases/tag/rMATS.3.2.5.GTEx.V8.final_matrices](https://github.com/adeslatt/sbas_test/releases/tag/rMATS.3.2.5.GTEx.V8.final_matrices). 

For using the [`ropensci/piggyback`](https://github.com/ropensci/piggyback) with private repositories, it is required that a `GITHUB_TOKEN` is stored as a variable in the R environment in which one is working. To generate such a token with sensible default permissions, the R package [usethis]() has a convenient function 

```R
# intall.packages("usethis")
usethis::browse_github_token()
```

This will redirect you to GitHub to create your own GitHub token. Once you have the token, you can use it to set up `.Renviron` by typing the following:

```R
Sys.setenv(GITHUB_TOKEN = "youractualtokenindoublequotes")
```

Then you are ready to use the function [`piggyback::pb_download()`](https://docs.ropensci.org/piggyback/reference/pb_download.html) to retrieve the `dge.tar.gz` that contains the topTable objects written as .csv file for all 46 examined GTEX tissue cohorts.

---

***NOTE***

Avoid using the `.token` argument to share your token directly in the function as you might forget and push your code, along with your private GITHUB_TOKEN to GitHub. If that happens by mistake, it is advised you invalidate the token that has been exposed by accessing [this link](https://github.com/settings/tokens) and clicking `Delete`.

---


In [13]:
# ijc
if (! (file.exists("../data/rmats_final.se.jc.ijc.txt.gz"))) {
    system("mkdir -p ../data", intern = TRUE)
    message("Fetching rmats_final.se.jc.ijc.txt.gz from GitHub ..")
    # Download archive from GitHub release with tag "dge"
    piggyback::pb_download(file = "rmats_final.se.jc.ijc.txt.gz",
                           dest = "../data",
                           repo = "adeslatt/sbas_test",
                           tag  = "rMATS.3.2.5.GTEx.V8.final_matrices",
                           show_progress = TRUE)
    message("Done!\n")
}

# sjc
if (! (file.exists("../data/rmats_final.se.jc.sjc.txt.gz"))) {
    system("mkdir -p ../data", intern = TRUE)
    message("Fetching rmats_final.se.jc.sjc.txt.gz from GitHub ..")
    # Download archive from GitHub release with tag "dge"
    piggyback::pb_download(file = "rmats_final.se.jc.sjc.txt.gz",
                           dest = "../data",
                           repo = "adeslatt/sbas_test",
                           tag  = "rMATS.3.2.5.GTEx.V8.final_matrices",
                           show_progress = TRUE)
    message("Done!\n")
}

#inc
if (! (file.exists("../data/rmats_final.se.jc.inc.txt.gz"))) {
    system("mkdir -p ../data", intern = TRUE)
    message("Fetching rmats_final.se.jc.inc.txt.gz from GitHub ..")
    # Download archive from GitHub release with tag "dge"
    piggyback::pb_download(file = "rmats_final.se.jc.inc.txt.gz",
                           dest = "../data",
                           repo = "adeslatt/sbas_test",
                           tag  = "rMATS.3.2.5.GTEx.V8.final_matrices",
                           show_progress = TRUE)
    message("Done!\n")
}

# SraRunTable.noCram.noExome.noWGS.totalRNA.txt
if (! (file.exists("../data/rmats_final.se.jc.inc.txt.gz"))) {
    system("mkdir -p ../data", intern = TRUE)
    message("Fetching rmats_final.se.jc.inc.txt.gz from GitHub ..")
    # Download archive from GitHub release with tag "dge"
    piggyback::pb_download(file = "SraRunTable.noCram.noExome.noWGS.totalRNA.txt.gz",
                           dest = "../data",
                           repo = "TheJacksonLaboratory/sbas",
                           tag  = "GTExV8.v1.0",
                           show_progress = TRUE)
    message("Done!\n")
}



if (file.exists("../data/rmats_final.se.jc.ijc.txt.gz")) {
    message("File rmats_final.se.jc.ijc.txt.gz already available in ../data/ !\n")
    }
if (file.exists("../data/rmats_final.se.jc.sjc.txt.gz")) {
    message("File rmats_final.se.jc.sjc.txt.gz already available in ../data/ !\n")
    }
if (file.exists("../data/rmats_final.se.jc.inc.txt.gz")) {
    message("File rmats_final.se.jc.inc.txt.gz already available in ../data/ !\n")
    }
if (file.exists("../data/SraRunTable.noCram.noExome.noWGS.totalRNA.txt.gz")) {
    message("File SraRunTable.noCram.noExome.noWGS.totalRNA.txt.gz already available in ../data/ !\n")
    }

File rmats_final.se.jc.ijc.txt.gz already available in ../data/ !


File rmats_final.se.jc.sjc.txt.gz already available in ../data/ !


File rmats_final.se.jc.inc.txt.gz already available in ../data/ !




## Modeling

This analysis uses edgeR.  From the documentation, it is important to note that normalization takes the form of correction factors that enter into the statistical model. Such correction factors are usually computed internally by edgeR functions, but it is also possible for a user to supply them. The correction factors may take the form of scaling factors for the library sizes, such as computed by calcNormFactors, which are then used to compute the effective library sizes. 

Alternatively, gene-specific correction factors can be entered into the glm functions of edgeR as offsets. In the latter case, the offset matrix will be assumed to account for all normalization issues, including sequencing depth and RNA composition.

Note that normalization in edgeR is model-based, and the original read counts are not themselves transformed. This means that users should not transform the read counts in any way before inputing them to edgeR. For example, users should not enter RPKM or FPKM val- ues to edgeR in place of read counts. Such quantities will prevent edgeR from correctly estimating the mean-variance relationship in the data, which is a crucial to the statistical strategies underlying edgeR. Similarly, users should not add artificial values to the counts before inputing them to edgeR.

edgeR is not designed to work with estimated expression levels, for example as might be output by Cufflinks. 
edgeR can work with expected counts as output by RSEM, but raw counts are still preferred. 

As instructed by the software, we are using the raw counts as provided by rMATS.  The raw counts we are using in the model are `ijc` and `sjc`, the sample specific raw read counts as they align to the junctions of the `included exon (ijc)` and the junctions of the `excluded or skipped exon (sjc)` respectively.



In [ ]:
ijc.iso.counts.mem <- data.table::fread("../data/rmats_final.se.jc.ijc.txt.gz") 
sjc.iso.counts.mem <- data.table::fread("../data/rmats_final.se.jc.sjc.txt.gz") 
inc.iso.counts.mem <- data.table::fread("../data/rmats_final.se.jc.inc.txt.gz")

meta.data<-read.csv('../data/SraRunTable.noCram.noExome.noWGS.totalRNA.txt',header=TRUE, stringsAsFactors=FALSE)
head(ijc.iso.counts.mem)
head(sjc.iso.counts.mem)
head(inc.iso.counts.mem)
head(meta.data)

## Synchronize metadata samples with ijc, sjc and inc samples

Keep only the runs that are in the ijc count list (assuming ijc and sjc are the same).  As well, name the rows with the junction id column and then make the matrix just about the counts.

In [ ]:
#dimensions before we make the changes.
dim(ijc.iso.counts.mem)
dim(sjc.iso.counts.mem)
dim(inc.iso.counts.mem)
dim(meta.data)

# the sample names are in the columns of both the ijc and the sjc matrices, these matrices have the identical column order)
keep.meta.data <- meta.data$Run %in% colnames(ijc.iso.counts.mem)
table(keep.meta.data)
reduced.meta.data <- meta.data[keep.meta.data==TRUE,]

## Construct the ijc, sjc and inc as data matrices
The Junction ID is encoded in the first column of the matrix.  We need to both preserve it (and it is unique) as well as remove it so we may do our calculations.

In [ ]:
# preserve junction id as rowname
rownames(ijc.iso.counts.mem) <- ijc.iso.counts.mem$ID
rownames(sjc.iso.counts.mem) <- sjc.iso.counts.mem$ID
rownames(inc.iso.counts.mem) <- inc.iso.counts.mem$ID

# and remove the id to have a data matrix
ijc.iso.counts.mem  <- ijc.iso.counts.mem[,-1]
sjc.iso.counts.mem  <- sjc.iso.counts.mem[,-1]
inc.iso.counts.mem  <- inc.iso.counts.mem[,-1]
dim(ijc.iso.counts.mem)
dim(sjc.iso.counts.mem)
dim(inc.iso.counts.mem)
dim(reduced.meta.data)

## Order ijc and sjc columns in the same order as the metadata Run order

Using tibble library, we can rearrange the columns as the column name.  

In [ ]:
meta.data.run.names  <- as.character(reduced.meta.data$Run)
ijc.iso.counts.mem2  <- as_tibble(ijc.iso.counts.mem)
sjc.iso.counts.mem2  <- as_tibble(sjc.iso.counts.mem)
inc.iso.counts.mem2  <- as_tibble(inc.iso.counts.mem)

ijc.iso.counts.mem2  <- ijc.iso.counts.mem2[,c(meta.data.run.names)]
sjc.iso.counts.mem2  <- sjc.iso.counts.mem2[,c(meta.data.run.names)]
inc.iso.counts.mem2  <- inc.iso.counts.mem2[,c(meta.data.run.names)]

Remove samples that match '11IL0' from the ijc, sjc and metadata files

In [ ]:
keep.meta.data <- (!grepl('11ILO',reduced.meta.data$Sample.Name))
table(keep.meta.data)
ijc.iso.counts.mem2 <-ijc.iso.counts.mem2 [                    ,keep.meta.data==TRUE]
sjc.iso.counts.mem2 <-sjc.iso.counts.mem2 [                    ,keep.meta.data==TRUE]
inc.iso.counts.mem2 <-inc.iso.counts.mem2 [                    ,keep.meta.data==TRUE]
reduced.meta.data   <-reduced.meta.data   [keep.meta.data==TRUE,                    ]
dim(ijc.iso.counts.mem2)
dim(sjc.iso.counts.mem2)
dim(inc.iso.counts.mem2)

### and focus on a single tissue

this will become a function so we can proceed on all the tissues

In [ ]:
tissue <- reduced.meta.data$body_site %in% 'Breast - Mammary Tissue'
table(tissue)

ijc.iso.counts.mem2 <-ijc.iso.counts.mem2 [                    ,tissue==TRUE]
sjc.iso.counts.mem2 <-sjc.iso.counts.mem2 [                    ,tissue==TRUE]
inc.iso.counts.mem2 <-inc.iso.counts.mem2 [                    ,tissue==TRUE]
reduced.meta.data   <-reduced.meta.data   [tissue==TRUE,                    ]
dim(ijc.iso.counts.mem2)
dim(sjc.iso.counts.mem2)
dim(inc.iso.counts.mem2)

### exploration of the details

For each sample, we have ijc and sjc count data and demographics of gender.
Our question is regarding the sex biased differences.
For each junction we have 8,000 samples with these count data.   The way to think about the model is that we have in fact for all of these junctions, these are our co-variates in this global transcriptomic model. 
For exon skipping events (SE), we have 42,611 non-zero junction IDs the (first dimension of the ijc and sjc cout table) for the skipped exon event for breast-Mammary Tissue, 191 individuals.  These are healthy individuals, and we are studying the impact of sex on the occurrence or non-occurance of specific alternative splicing events.   We explore the information we ahve about these junctions and create a construct, as_event, which accounts for the junction under exploration.

In [ ]:
ijc      <- as.data.frame(ijc.iso.counts.mem2)
sjc      <- as.data.frame(sjc.iso.counts.mem2)
ijc      <- data.matrix(ijc)
sjc      <- data.matrix(sjc)


## Exploring the ijc and sjc Count data 

We have two counts that are in many ways two sides of the same coin.  Both our the observational output and we wish to see how robust each are in their ability to separate out the samples to provide for us differentially expressed isoform events as measured by their counts.   Each junction is in a manner a specific marker to specific isoform events that may or may not be shared between the genders.   If there is significant results, then this is indicative of the separation achieved by isoform specific differentiation.   In our model we will use these in combination, it is important to see if they will yield the results we are looking for.

### IJC

Exon included junction counts -- duplicate correlation is actually 2 (there are 2 exons for each included exon) the results provide robust separation up until around 550.  Also important to filter low expression counts

In [ ]:
sex       <- factor(reduced.meta.data$sex,levels=c('male','female'))

design    <- model.matrix ( ~ sex)
y <- DGEList(counts=ijc, group = sex)
y <- calcNormFactors(y, method="upperquartile")

sex       <- factor(reduced.meta.data$sex,levels=c('male','female'))

design    <- model.matrix ( ~ sex)

y_voom <- voom (y, design=design, plot=TRUE )

Gender <- substring(sex,1,1)

plotMDS(y, labels=Gender, top=500, col=ifelse(Gender=="m","blue","red"), 
        gene.selection="common")
plotMDS(y_voom, labels=Gender, top=500, col=ifelse(Gender=="m","blue","red"), 
        gene.selection="common")
plotMDS(y, labels=Gender, top=1500, col=ifelse(Gender=="m","blue","red"), 
        gene.selection="common")
plotMDS(y_voom, labels=Gender, top=1500, col=ifelse(Gender=="m","blue","red"), 
        gene.selection="common")


### IJC Plots saved as PDFs

In [ ]:
pdf("../pdf/BreastMammaryTissue_ijc_voom.pdf")
y_voom <- voom (y, design=design, plot = TRUE, save.plot = TRUE)
dev.off()

Gender <- substring(sex,1,1)

pdf("../pdf/BreastMammaryTissue_ijc_MDSplot_top500.pdf")
plotMDS(y, labels=Gender, top=500, col=ifelse(Gender=="m","blue","red"), 
        gene.selection="common")
dev.off()

pdf("../pdf/BreastMammaryTissue_ijc_voom_MDSplot_top500.pdf")
plotMDS(y_voom, labels=Gender, top=500, col=ifelse(Gender=="m","blue","red"), 
        gene.selection="common")
dev.off()

pdf("../pdf/BreastMammaryTissue_ijc_MDSplot_top1500.pdf")
plotMDS(y, labels=Gender, top=1500, col=ifelse(Gender=="m","blue","red"), 
        gene.selection="common")
dev.off()

pdf("../pdf/BreastMammaryTissue_ijc_voom_MDSplot_top1500.pdf")
plotMDS(y_voom, labels=Gender, top=1500, col=ifelse(Gender=="m","blue","red"), 
        gene.selection="common")
dev.off()



### SJC
sjc counts are skipped exon junction counts -- ijc counts hold together for the top 1000, the skipped exon junction counts fall appart
at a lower number -- around 200!  separation fails between the genders at 250

In [ ]:
sex       <- factor(reduced.meta.data$sex,levels=c('male','female'))
design    <- model.matrix(~ sex)
y <- DGEList(counts=sjc, group = sex)
y <- calcNormFactors(y, method="upperquartile")

sex       <- factor(reduced.meta.data$sex,levels=c('male','female'))

design    <- model.matrix ( ~ sex)

y_voom <- voom (y, design=design, plot = TRUE)

Gender <- substring(sex,1,1)

plotMDS(y, labels=Gender, top=250, col=ifelse(Gender=="m","blue","red"), 
        gene.selection="common")
plotMDS(y_voom, labels=Gender, top=250, col=ifelse(Gender=="m","blue","red"), 
        gene.selection="common")
plotMDS(y, labels=Gender, top=600, col=ifelse(Gender=="m","blue","red"), 
        gene.selection="common")
plotMDS(y_voom, labels=Gender, top=600, col=ifelse(Gender=="m","blue","red"), 
        gene.selection="common")


### SJC Plots saved as PDFs

In [ ]:
pdf("../pdf/BreastMammaryTissue_sjc_voom.pdf")
y_voom <- voom (y, design=design, plot = TRUE, save.plot = TRUE)
dev.off()

Gender <- substring(sex,1,1)

pdf("../pdf/BreastMammaryTissue_sjc_MDSplot_top250.pdf")
plotMDS(y, labels=Gender, top=250, col=ifelse(Gender=="m","blue","red"), 
        gene.selection="common")
dev.off()

pdf("../pdf/BreastMammaryTissue_sjc_voom_MDSplot_top250.pdf")
plotMDS(y_voom, labels=Gender, top=250, col=ifelse(Gender=="m","blue","red"), 
        gene.selection="common")
dev.off()


pdf("../pdf/BreastMammaryTissue_sjc_MDSplot_top600.pdf")
plotMDS(y, labels=Gender, top=600, col=ifelse(Gender=="m","blue","red"), 
        gene.selection="common")
dev.off()

pdf("../pdf/BreastMammaryTissue_sjc_voom_MDSplot_top600.pdf")
plotMDS(y_voom, labels=Gender, top=600, col=ifelse(Gender=="m","blue","red"), 
        gene.selection="common")
dev.off()


## Differential analysis as_event:ijc 

Differential Analysis (DE) was performed using voom (Law et.al., 2014) to transform junction counts (reads that were aligned to junctions when an exon is included - ijc, and reads that were aligned to junctions when the exon is excluded - sjc) with associated precision weights, followed by linear modeling and empirical Bayes procedure using limma.    In each tissue, the following linear regression model was used to detec secually dimorphic alternative splicing event expression: 

           y = B0 + B1 sex + epsilon (error)
           

where y is the included exon junction count expression; sex denotes the reported sex of the subject.

In [ ]:
sex       <- reduced.meta.data$sex

design    <- model.matrix( ~ sex )

colnames(design) <- c("intercept","sex")

dim(ijc)
table(sex)
head(design)

y_ijc <- DGEList(counts=ijc, group = sex)
y_ijc <- calcNormFactors(y_ijc, method="upperquartile")

y_ijc_voom <- voom (y_ijc, design=design)

Gender <- substring(sex,1,1)

plotMDS(y_ijc, labels=Gender, top=1000, col=ifelse(Gender=="m","blue","red"), 
        gene.selection="common")

plotMDS(y_ijc_voom, labels=Gender, top=1000, col=ifelse(Gender=="m","blue","red"), 
        gene.selection="common")

fit_ijc <- lmFit(y_ijc_voom, design)
fit_ijc <- eBayes(fit_ijc)

ijc_sex_results          <- topTable(fit_ijc, coef='sex', number=nrow(y_voom))
ijc_sex_results_refined  <- ijc_sex_results$adj.P.Val < 0.05 & abs(ijc_sex_results$logFC) > 1.5

table(ijc_sex_results_refined)

## Differential analysis as_event:sjc 

Differential Analysis (DE) was performed using voom (Law et.al., 2014) to transform junction counts (reads that were aligned to junctions when an exon is included - ijc, and reads that were aligned to junctions when the exon is excluded - sjc) with associated precision weights, followed by linear modeling and empirical Bayes procedure using limma.    In each tissue, the following linear regression model was used to detec secually dimorphic alternative splicing event expression: 

           y = B0 + B1 sex + epsilon (error)
           

where y is the excluded exon junction count (sjc) expression; sex denotes the reported sex of the subject.

In [ ]:
sex       <- reduced.meta.data$sex

design    <- model.matrix( ~ sex )

colnames(design) <- c("intercept","sex")

dim(sjc)
table(sex)
head(design)

y_sjc <- DGEList(counts=sjc, group = sex)
y_sjc <- calcNormFactors(y_sjc, method="upperquartile")

y_sjc_voom <- voom (y_sjc, design=design)

Gender <- substring(sex,1,1)

plotMDS(y_sjc, labels=Gender, top=500, col=ifelse(Gender=="m","blue","red"), 
        gene.selection="common")

plotMDS(y_sjc_voom, labels=Gender, top=500, col=ifelse(Gender=="m","blue","red"), 
        gene.selection="common")

fit_sjc <- lmFit(y_sjc_voom, design)
fit_sjc <- eBayes(fit_sjc)

sjc_sex_results          <- topTable(fit_sjc, coef='sex', number=nrow(y_voom))
sjc_sex_results_refined  <- sjc_sex_results$adj.P.Val < 0.05 & abs(sjc_sex_results$logFC) > 1.5

table(sjc_sex_results_refined)

## Differential analysis as_event (combined ijc and sjc)

Differential Analysis (DE) was performed using voom (Law et.al., 2014) to transform junction counts (reads that were aligned to junctions when an exon is included - ijc, and reads that were aligned to junctions when the exon is excluded - sjc) with associated precision weights, followed by linear modeling and empirical Bayes procedure using limma.    In each tissue, the following linear regression model was used to detec secually dimorphic alternative splicing event expression: 

           y = B0 + B1 sex + B2 as_event + B3 sex*as_event + epsilon (error)
           
         alt_y = B0 + B1 sample + B2 sex + B3 as_event + B4 sex*as_event + epsilon (error)

where y is the alternative splicing event expression; sex denotes the reported sex of the subject, as_event represents the specific alternative splicing event - either included exon junction counts or skipped exon junction counts and their interaction terms.  Modeling the donor and tissue effect as a random effect, for simplicity, we call this sample.


In [ ]:
sample    <- c(rep(colnames(ijc.iso.counts.mem2),2))

ijc_names <- as.character(colnames(ijc))
sjc_names <- as.character(colnames(sjc))
ijc_names <- paste0(ijc_names,"-ijc")
sjc_names <- paste0(sjc_names,"-sjc")
colnames(ijc) <- ijc_names
colnames(sjc) <- sjc_names

as_matrix <- cbind(ijc,sjc)

sex       <- c(rep(reduced.meta.data$sex,2))
sex       <- factor(sex, levels=c('male','female'))

as_event  <- c(rep("ijc",dim(ijc)[2]), rep("sjc", dim(sjc)[2]))
as_event  <- factor(as_event, levels=c("ijc", "sjc"))

# in this design, we are modeling sample as a random effect - but we could also model it in the design
design    <- model.matrix( ~ sex + as_event + sex*as_event)

colnames(design) <- c("intercept","sex","as_event","sex*as_event")

length(sample)
dim(as_matrix)
table(sex)
table(as_event)
head(design)


### Voom, limma's lmFit and eBayes

The final factor to model is to capture the impact of the sample (which is a combination of the donor and the tissue - in this single tissue analysis -- it is just the donor) - given that we have so many 

In [ ]:
y <- DGEList(counts=as_matrix, group = sex)
y <- calcNormFactors(y, method="upperquartile")
y_no_dup_voom <- voom (y, design=design)

In [ ]:
dup_cor <- duplicateCorrelation(y_no_dup_voom$E, design=design, ndups=2, block=sample, weights=y$samples$norm.factors)
dup_cor$consensus.correlation

In [ ]:
y_dup_voom <- voom (y, design=design, plot = TRUE, correlation = dup_cor$consensus_correlation)

In [ ]:
Gender <- substring(sex[1:191],1,1)

plotMDS(y_no_dup_voom[,c(1:191)], labels=Gender, top=500, col=ifelse(Gender=="m","blue","red"), 
        gene.selection="common")

In [ ]:
plotMDS(y_dup_voom[,c(1:191)], labels=Gender, top=500, col=ifelse(Gender=="m","blue","red"), 
        gene.selection="common")

In [ ]:
pdf ("../pdf/BreastMammaryTissue_y_no_dup_voom_top_500.pdf")
plotMDS(y_no_dup_voom[,c(1:191)], labels=Gender, top=500, col=ifelse(Gender=="m","blue","red"), 
        gene.selection="common")
dev.off()
pdf ("../pdf/BreastMammaryTissue_y_dup_voom_top_500.pdf")
plotMDS(y_dup_voom[,c(1:191)], labels=Gender, top=500, col=ifelse(Gender=="m","blue","red"), 
        gene.selection="common")
dev.off()

### linear fit with eBayes

With a correlation of 0.19 the two measures ijc and sjc are weakly correlated per donor. Which is to be expected, as for any single donor one would expect that the inclusion of an exon would result in the increase in the inclusion junction counts (reads aligned to the junctions associated with the included exon) and not expect that that same donor would have also evidence of excluded exon (where the counts would be to the junction revealed by the exon being excluded).   However, some amount of correlation might be here because of a haplotype affect and the location being on one allele maternally inherited versions another allele paternally inherited. 

Nevertheless, we include the duplicate correlation as calculated in our analysis.

In [ ]:
fit <- lmFit(y_dup_voom, design)
fit <- eBayes(fit, robust = TRUE)

In [ ]:
sex_as_events_results         <- topTable(fit, coef="sex*as_event", number=nrow(y_no_dup_voom))
sex_as_events_results_refined <- sex_as_events_results$adj.P.Val < 0.05 & abs(sex_as_events_results$logFC) > 1.5

sex_results                   <- topTable(fit, coef="sex", number=nrow(y_no_dup_voom))
sex_results_refined           <- sex_results$adj.P.Val < 0.05 & abs(sex_results$logFC) > 1.5

table(sex_as_events_results_refined)
table(sex_results_refined)

#### fromGTF

rMATS analyzes the gtf file and determines the junctions, identifying 10 matrices in all.  The fromGTF.SE.txt contains the annotation information and we can use this information to estimate the duplicate correlation effect from counting based upon the same gene with our ijc counts.  we will use this information, to obtain the gene information.


In [ ]:
ijc_sex_results          <- topTable(fit_ijc, coef='sex', number=nrow(y_ijc_voom))
sjc_sex_results          <- topTable(fit_sjc, coef='sex', number=nrow(y_sjc_voom))

In [ ]:
head(ijc_sex_results)
head(sjc_sex_results)
head(sex_as_events_results)
head(sex_results)

In [ ]:
fromGTF.SE <- read.table("../data/fromGTF.SE.txt", header=TRUE)
head(fromGTF.SE)
genes <- factor(fromGTF.SE$geneSymbol)
length(levels(genes))

In [ ]:
# index to the annotations - these are the Junction IDs
sex_as_events_rnResults <- rownames(sex_as_events_results)
sex_rnResults           <- rownames(sex_results)
ijc_sex_rnResults       <- rownames(ijc_sex_results)
sjc_sex_rnResults       <- rownames(sjc_sex_results)
head(sex_as_events_rnResults)
head(ijc_sex_rnResults)
head(sjc_sex_rnResults)
head(sex_rnResults)
head(fromGTF.SE[sex_as_events_rnResults,])

In [ ]:
# use the junctionIDs to get the annotations
sex_as_events_resultsAnnotations      <- fromGTF.SE[sex_as_events_rnResults,]
sex_resultsAnnotations                <- fromGTF.SE[sex_rnResults,]
ijc_sex_resultsAnnotations            <- fromGTF.SE[ijc_sex_rnResults,]
sjc_sex_resultsAnnotations            <- fromGTF.SE[sjc_sex_rnResults,]
head(sex_as_events_resultsAnnotations)
head(sex_resultsAnnotations)
head(ijc_sex_resultsAnnotations)
head(sjc_sex_resultsAnnotations)

In [ ]:
sex_as_events_results_refinedAnnotations<- sex_as_events_resultsAnnotations[sex_as_events_results_refined==TRUE,]
sex_results_refinedAnnotations          <- sex_resultsAnnotations          [sex_results_refined          ==TRUE,]
ijc_sex_results_refinedAnnotations      <- ijc_sex_resultsAnnotations      [ijc_sex_results_refined      ==TRUE,]
sjc_sex_results_refinedAnnotations      <- sjc_sex_resultsAnnotations      [sjc_sex_results_refined      ==TRUE,]
head(sex_as_events_results_refinedAnnotations)
head(sex_results_refinedAnnotations)
head(ijc_sex_results_refinedAnnotations)
head(sjc_sex_results_refinedAnnotations)

In [ ]:

# geneSymbols are in the annotations
sex_as_events_geneSymbols         <- sex_as_events_resultsAnnotations$geneSymbol
sex_geneSymbols                   <- sex_resultsAnnotations$geneSymbol
ijc_sex_geneSymbols               <- ijc_sex_resultsAnnotations$geneSymbol
sjc_sex_geneSymbols               <- sjc_sex_resultsAnnotations$geneSymbol

sex_as_events_refined_geneSymbols <- sex_as_events_results_refinedAnnotations$geneSymbol
sex_refined_geneSymbols           <- sex_results_refinedAnnotations$geneSymbol
ijc_sex_refined_geneSymbols       <- ijc_sex_results_refinedAnnotations$geneSymbol
sjc_sex_refined_geneSymbols       <- sjc_sex_results_refinedAnnotations$geneSymbol

# adjust the rownames to be the geneSymbols rather than junction IDs
sex_as_events_results_rn   <- paste(sex_as_events_geneSymbols, sex_as_events_rnResults, sep="-")
sex_results_rn             <- paste(sex_geneSymbols,           sex_rnResults, sep="-")
ijc_sex_results_rn         <- paste(ijc_sex_geneSymbols,       ijc_sex_rnResults, sep="-")
sjc_sex_results_rn         <- paste(sjc_sex_geneSymbols,       sjc_sex_rnResults, sep="-")

head(sex_as_events_results_rn)
head(sex_results_rn)
head(ijc_sex_results_rn)
head(sjc_sex_results_rn)


In [ ]:
rownames(sex_as_events_results) <- sex_as_events_results_rn
rownames(sex_results)           <- sex_results_rn
rownames(ijc_sex_results)       <- ijc_sex_results_rn
rownames(sjc_sex_results)       <- sjc_sex_results_rn

In [ ]:
sex_as_events_filename         = paste(paste('../data/BreastMammaryTissue',collapse='.'),'_DGE_sex_as_events.csv',sep='')
sex_filename                   = paste(paste('../data/BreastMammaryTissue',collapse='.'),'_DGE_sex.csv',sep='')
ijc_sex_filename               = paste(paste('../data/BreastMammaryTissue',collapse='.'),'_DGE_ijc_sex.csv',sep='')
sjc_sex_filename               = paste(paste('../data/BreastMammaryTissue',collapse='.'),'_DGE_sjc_sex.csv',sep='')

sex_as_events_refined_filename = paste(paste('../data/BreastMammaryTissue',collapse='.'),'_DGE_sex_as_events_refined.csv',sep='')
sex_refined_filename           = paste(paste('../data/BreastMammaryTissue',collapse='.'),'_DGE_sex_refined.csv',sep='')
ijc_sex_refined_filename       = paste(paste('../data/BreastMammaryTissue',collapse='.'),'_DGE_ijc_sex_refined.csv',sep='')
sjc_sex_refined_filename       = paste(paste('../data/BreastMammaryTissue',collapse='.'),'_DGE_sjc_sex_refined.csv',sep='')

sex_as_events_genesFilename    = paste(paste('../data/BreastMammaryTissue',collapse='.'),'_sex_as_events_universe.txt',sep='')
sex_genesFilename              = paste(paste('../data/BreastMammaryTissue',collapse='.'),'_sex_universe.txt',sep='')
ijc_sex_genesFilename          = paste(paste('../data/BreastMammaryTissue',collapse='.'),'_ijc_sex_universe.txt',sep='')
sjc_sex_genesFilename          = paste(paste('../data/BreastMammaryTissue',collapse='.'),'_sjc_sex_universe.txt',sep='')

sex_as_events_refined_genesFilename = paste(paste('../data/BreastMammaryTissue',collapse='.'),'_sex_as_events_gene_set.txt',sep='')
sex_refined_genesFilename           = paste(paste('../data/BreastMammaryTissue',collapse='.'),'_sex_gene_set.txt',sep='')
ijc_sex_refined_genesFilename       = paste(paste('../data/BreastMammaryTissue',collapse='.'),'_ijc_sex_gene_set.txt',sep='')
sjc_sex_refined_genesFilename       = paste(paste('../data/BreastMammaryTissue',collapse='.'),'_sjc_sex_gene_set.txt',sep='')


In [ ]:
write.table(sex_as_events_results, file = sex_as_events_filename, row.names = T, col.names = T, quote = F, sep = ",")
write.table(sex_results,           file = sex_filename          , row.names = T, col.names = T, quote = F, sep = ",")
write.table(ijc_sex_results,       file = ijc_sex_filename      , row.names = T, col.names = T, quote = F, sep = ",")
write.table(sjc_sex_results,       file = sjc_sex_filename      , row.names = T, col.names = T, quote = F, sep = ",")

write.table(sex_as_events_results[sex_as_events_results_refined,], file = sex_as_events_refined_filename, row.names = T, col.names = T, quote = F, sep = ",")
write.table(sex_results          [sex_results_refined          ,], file = sex_refined_filename          , row.names = T, col.names = T, quote = F, sep = ",")
write.table(ijc_sex_results      [ijc_sex_results_refined      ,], file = ijc_sex_refined_filename      , row.names = T, col.names = T, quote = F, sep = ",")
write.table(sjc_sex_results      [sjc_sex_results_refined      ,], file = sjc_sex_refined_filename      , row.names = T, col.names = T, quote = F, sep = ",")

write.table(sex_as_events_geneSymbols, file = sex_as_events_genesFilename, row.names = F, col.names = F, quote = F, sep = ",")
write.table(sex_geneSymbols,           file = sex_genesFilename          , row.names = F, col.names = F, quote = F, sep = ",")
write.table(ijc_sex_geneSymbols,       file = ijc_sex_genesFilename      , row.names = F, col.names = F, quote = F, sep = ",")
write.table(sjc_sex_geneSymbols,       file = sjc_sex_genesFilename      , row.names = F, col.names = F, quote = F, sep = ",")

write.table(sex_as_events_refined_geneSymbols,file = sex_as_events_refined_genesFilename, row.names = F, col.names = F, quote = F, sep = ",")
write.table(sex_refined_geneSymbols,          file = sex_refined_genesFilename          , row.names = F, col.names = F, quote = F, sep = ",")
write.table(ijc_sex_refined_geneSymbols,      file = ijc_sex_refined_genesFilename      , row.names = F, col.names = F, quote = F, sep = ",")
write.table(sjc_sex_refined_geneSymbols,      file = sjc_sex_refined_genesFilename      , row.names = F, col.names = F, quote = F, sep = ",")


## Metadata

For replicability and reproducibility purposes, we also print the following metadata:

1. Checksums of **'artefacts'**, files generated during the analysis and stored in the folder directory **`data`**
2. List of environment metadata, dependencies, versions of libraries using `utils::sessionInfo()` and [`devtools::session_info()`](https://devtools.r-lib.org/reference/session_info.html)

### 1. Checksums with the sha256 algorithm

In [ ]:
notebookid   = "BreastMammaryTissueJunctionAnalysis"

message("Generating sha256 checksums of the artefacts in the `..data/` directory .. ")
system(paste0("cd ../data && find . -type f -exec sha256sum {} \\;  >  ../metadata/", notebookid, "_sha256sums.txt"), intern = TRUE)
message("Done!\n")

data.table::fread(paste0("../metadata/", notebookid, "_sha256sums.txt"), header = FALSE, col.names = c("sha256sum", "file"))

### 2. Libraries metadata

In [ ]:
dev_session_info   <- devtools::session_info()
utils_session_info <- utils::sessionInfo()

message("Saving `devtools::session_info()` objects in ../metadata/devtools_session_info.rds  ..")
saveRDS(dev_session_info, file = paste0("../metadata/", notebookid, "_devtools_session_info.rds"))
message("Done!\n")

message("Saving `utils::sessionInfo()` objects in ../metadata/utils_session_info.rds  ..")
saveRDS(utils_session_info, file = paste0("../metadata/", notebookid ,"_utils_info.rds"))
message("Done!\n")

dev_session_info$platform
dev_session_info$packages[dev_session_info$packages$attached==TRUE, ]